In [11]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

from tensorflow import keras


rng = 69

In [12]:
df = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Transformed.csv")
#df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_All_Games_Transformed.csv")

In [13]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away','goalsTotal_Away'])

In [14]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [15]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [16]:
corr_matrix['win_or_lose_Home'].sort_values(ascending=False)

win_or_lose_Home             1.000000
seasonPointsPerGame_Home     0.276991
fenwickPercentageAvg_Home    0.145742
pointsFromGameAvg_Home       0.094528
goalsAgainstAvg_Away         0.080603
hitsDiffAvg_Away             0.075956
goalsForAvg_Home             0.069071
reboundsForAvg_Home          0.051855
penaltiesAgainstTotal        0.024796
goalsTotal_Home              0.002599
penaltiesForTotal           -0.016065
hitsDiffAvg_Home            -0.026004
reboundsForAvg_Away         -0.058582
goalsForAvg_Away            -0.076319
goalsAgainstAvg_Home        -0.084765
pointsFromGameAvg_Away      -0.101494
fenwickPercentageAvg_Away   -0.154375
seasonPointsPerGame_Away    -0.272244
Name: win_or_lose_Home, dtype: float64

In [17]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [18]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [21]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('goalsTotal_Home', axis=1)
nhl_labels = train_set['goalsTotal_Home'].copy()

X_train = train_set.drop('goalsTotal_Home', axis=1)
Y_train = train_set['goalsTotal_Home'].copy()

X_valid = train_set.drop('goalsTotal_Home', axis=1)
Y_valid = train_set['goalsTotal_Home'].copy()

X_test = test_set.drop('goalsTotal_Home', axis=1)
Y_test = test_set['goalsTotal_Home'].copy()

df_Final = df_Final.drop('goalsTotal_Home', axis=1)

In [23]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [27]:
# set epochs to use same num in all calls
n_epochs = 50

In [29]:
def build_model(learning_rate = 0.00335195324238683, n_hidden=3, n_neurons_hidden=10, n_input_neurons=17, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(n_input_neurons,shape=input_shape))
    if n_hidden > 0:
        for _ in range(n_hidden):
            model.add(keras.layers.Dropout(rate=0.2))
            model.add(keras.layers.Dense(n_neurons_hidden,activation='selu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',optimizer= "sgd")
    return model

In [31]:
# Build model using best params from hypertuning

model = build_model

In [46]:
learning_rate = 0.00335195324238683
n_hidden=3
n_neurons_hidden=10
n_input_neurons=17
input_shape=[X_train.shape[1]]

model = keras.models.Sequential([
    keras.layers.InputLayer(shape=input_shape),
    keras.layers.Dense(n_neurons_hidden, activation='relu', input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss='mean_squared_error',optimizer= "sgd")

C:\Users\zchodan\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [54]:
import tensorflow as tf

tf.random.set_seed(42)
# Using a Callback to run all epochs and only save the best one

checkpoint_cb = keras.callbacks.ModelCheckpoint("Regression_Sequential.keras", save_best_only = True)
history = model.fit(X_train_prepared, Y_train, 
              validation_data = (X_valid_prepared, Y_valid),
              callbacks=[checkpoint_cb],epochs=30)

Epoch 1/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 16.2461 - val_loss: 6.0893
Epoch 2/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.9845 - val_loss: 5.7138
Epoch 3/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.6491 - val_loss: 5.5915
Epoch 4/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5285 - val_loss: 5.5334
Epoch 5/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.4726 - val_loss: 5.4983
Epoch 6/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.4363 - val_loss: 5.4754
Epoch 7/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.4141 - val_loss: 5.4605
Epoch 8/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.3991 - val_loss: 5.4472
Epoch 9/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.3858 - val_loss: 5.4355
Epoch 10/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.3723 - val_loss: 5.4260
Epoch 11/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.3600 - val_loss: 5.4162
Epoch 12/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste

In [77]:
X_train_prepared.itemsize

8

In [56]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold


kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
Y_pred = cross_val_predict(keras_clf, X_train_prepared, Y_train, cv=kfold)

from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, Y_pred)
# [Predict Loss - Actual Loss  |  Predict Win - Actual Loss]
# [Predict Loss - Actual Win   |  Predict Win - Actual Win]

C:\Users\zchodan\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


TypeError: InputLayer.__init__() got multiple values for argument 'shape'

In [61]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('When claiming is a win, correct % is ', precision_score(Y_train, Y_pred))
print('Detected % of wins' ,recall_score(Y_train, Y_pred))
print('F1 score is ', f1_score(Y_train, Y_pred))
print('Training Accuracy score is ', accuracy_score(Y_train, Y_pred))


NameError: name 'Y_pred' is not defined

In [63]:
# Train Accuracy
Y_train_pred = model.predict(X_train_prepared)
print(accuracy_score(Y_train, Y_train_pred))

Y_test_pred = model.predict(X_test_prepared)
print(f'Test score: {accuracy_score(Y_test, Y_test_pred)}')

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [67]:
df_Predict = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Predict.csv")
#df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_All_Games_Predict.csv")

In [71]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,model.predict(X_real_predict))).T

InvalidArgumentError: Graph execution error:

Detected at node sequential_2_1/dense_2_1/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\zchodan\AppData\Local\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\zchodan\AppData\Local\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\zchodan\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "C:\Users\zchodan\AppData\Local\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\zchodan\AppData\Local\anaconda3\Lib\asyncio\windows_events.py", line 322, in run_forever

  File "C:\Users\zchodan\AppData\Local\anaconda3\Lib\asyncio\base_events.py", line 641, in run_forever

  File "C:\Users\zchodan\AppData\Local\anaconda3\Lib\asyncio\base_events.py", l

In [ ]:
X_real_predict